In [3]:
import pandas as pd
import numpy as np
import re


In [5]:
# Load data
Buzzfeed_clean = pd.read_csv('data/Buzzfeed_data.csv')
top1 = pd.read_csv('data/top1_fake_title.csv')
top2 = pd.read_csv('data/top2_real_title.csv')
top3 = pd.read_csv('data/top3_fake_body.csv')
top4 = pd.read_csv('data/top4_real_body.csv')

In [7]:
# Convert top words to list
fake_title_words = top1['word'].tolist()
real_title_words = top2['word'].tolist()
fake_body_words = top3['word'].tolist()
real_body_words = top4['word'].tolist()

# Function to count matches of any word in text
def count_words_in_text(text, word_list):
    if isinstance(text, str):
        return sum(bool(re.search(rf'\b{word}\b', text)) for word in word_list)
    else:
        return 0

# Helper to count occurrences of a word
def count_word(text, word):
    if isinstance(text, str):
        return len(re.findall(rf'\b{word}\b', text))
    else:
        return 0


In [9]:
Buzzfeed_clean

,title,text,source,news_type,contain_movies,contain_images
0,Another Terrorist Attack in NYC…Why Are we STI...,"On Saturday, September 17 at 8:30 pm EST, an e...",http://eaglerising.com,Real,0,1
1,"Donald Trump: Drugs a 'Very, Very Big Factor' ...",Less than a day after protests over the police...,http://abcn.ws,Real,0,1
2,"Obama To UN: ‘Giving Up Liberty, Enhances Secu...","Obama To UN: ‘Giving Up Liberty, Enhances Secu...",http://rightwingnews.com,Real,1,1
3,Trump vs. Clinton: A Fundamental Clash over Ho...,Getty Images Wealth Of Nations Trump vs. Clint...,http://politi.co,Real,0,1
4,"President Obama Vetoes 9/11 Victims Bill, Sett...",President Obama today vetoed a bill that would...,http://abcn.ws,Real,0,1
...,...,...,...,...,...,...
177,Hillary’s TOP Donor Country Just Auctioned Off...,Hillary’s TOP Donor Country Just Auctioned Off...,http://rightwingnews.com,Fake,0,1
178,Cavuto Just Exposed Lester Holt's Lies During ...,Advertisement - story continues below\n\nThe f...,http://conservativetribune.com,Fake,1,1
179,"The AP, In 2004, Said Your Boy Obama Was BORN ...",Well THAT’S Weird. If the Birther movement is ...,http://clashdaily.com,Fake,0,1
180,People Noticed Something Odd About Hillary's O...,\n\nThere’s a lot to be discussed about last n...,http://www.thepoliticalinsider.com,Fake,0,1


In [ ]:
# Create fake title word columns
for word in fake_title_words:
    col_name = f"fake_title_{word}"
    Buzzfeed_clean[col_name] = Buzzfeed_clean['title'].apply(lambda x: count_word(x, word))

# Create real title word columns
for word in real_title_words:
    col_name = f"real_title_{word}"
    Buzzfeed_clean[col_name] = Buzzfeed_clean['title'].apply(lambda x: count_word(x, word))
    
##########    

# Create fake body word columns
for word in fake_body_words:
    col_name = f"fake_body_{word}"
    Buzzfeed_clean[col_name] = Buzzfeed_clean['text'].apply(lambda x: count_word(x, word))

# Create real body word columns
for word in real_body_words:
    col_name = f"real_body_{word}"
    Buzzfeed_clean[col_name] = Buzzfeed_clean['text'].apply(lambda x: count_word(x, word))

# Show new columns
Buzzfeed_clean.head(3)

,title,text,source,news_type,contain_movies,contain_images,fake_title_hillari,fake_title_clinton,fake_title_obama,fake_title_freedom,...,real_body_presidenti,real_body_american,real_body_nation,real_body_year,real_body_make,real_body_dont,real_body_polic,real_body_news,real_body_need,real_body_offic
0,Another Terrorist Attack in NYC…Why Are we STI...,"On Saturday, September 17 at 8:30 pm EST, an e...",http://eaglerising.com,Real,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,3,0
1,"Donald Trump: Drugs a 'Very, Very Big Factor' ...",Less than a day after protests over the police...,http://abcn.ws,Real,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,"Obama To UN: ‘Giving Up Liberty, Enhances Secu...","Obama To UN: ‘Giving Up Liberty, Enhances Secu...",http://rightwingnews.com,Real,1,1,0,0,0,0,...,0,0,1,1,2,0,0,0,2,0


182